In [1]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import math
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
def autoCymax(inventory,realTimeInventory):
    orig_df = pd.read_csv(inventory)

    realTime_df = pd.read_csv(realTimeInventory)
    realTime_df = realTime_df[['Name', 'Location Available', 'Location']]
    realTime_df = realTime_df[realTime_df['Location'] == 'KFHQ']
    realTime_df['Name'] = realTime_df['Name'].astype(str)
    orig_df['SKU'] = orig_df['SKU'].astype(str)
    realTime_df.rename(columns = {'Name':'SKU'}, inplace = True)
    new_df = pd.merge(orig_df, realTime_df, on = 'SKU', how = 'left')
    chino = new_df['Quantity On Hand']
    supply = new_df['Location Available']

    avaiabledate = new_df['Next Availability Date']


    given_date = date.today()
    future_date = given_date+ relativedelta(months=5)
    future_date=future_date.strftime('%m/%d/%Y')
    #future_date = str(future_date).replace('-','/')
    #somehow we going into all the rows

    for x in range(len(chino)):
        if math.isnan(supply[x]) == False:

            if supply[x] <= 40:
                chino[x] = 0
                avaiabledate[x] = future_date
            elif supply[x] <= 60:
                chino[x] = 10
            elif supply[x] <= 100:
                chino[x] = 20
            else:
                chino[x] = int(supply[x] * .3)
        else:
            chino[x] = 0;

    new_df['Quantity On Hand'] = chino
    new_df = new_df.iloc[:,0:4]
    new_df.to_excel('.\\CyMax\\CyMax' + str(date.today()) + '.xlsx', index = False)
    new_df.to_csv(homePop, index = False)

In [3]:
def automateHomePop(inventory, realTimeInventory):
    orig_df = pd.read_csv(inventory)

    realTime_df = pd.read_csv(realTimeInventory)
    display(realTime_df)
    realTime_df = realTime_df[['Name', 'Location Available', 'Location']]
    realTime_df = realTime_df[realTime_df['Location'] == 'KFHQ']
    realTime_df['Name'] = realTime_df['Name'].astype(str)
    orig_df['SKU'] = orig_df['SKU'].astype(str)
    realTime_df.rename(columns = {'Name':'SKU'}, inplace = True)
    new_df = pd.merge(orig_df, realTime_df, on = 'SKU', how = 'left')
    chino = new_df['Chino']
    supply = new_df['Location Available']
    #somehow we going into all the rows

    for x in range(len(chino)):
        if math.isnan(supply[x]) == False:

            if supply[x] <= 40:
                chino[x] = 0
            elif supply[x] <= 60:
                chino[x] = 10
            elif supply[x] <= 100:
                chino[x] = 20
            else:
                chino[x] = int(supply[x] * .3)
        else:
            chino[x] = 0;
    new_df['Chino'] = chino
    new_df = new_df.iloc[:,0:8]
    new_df.to_excel('.\\Homepop\\revisedHomePOP_' + str(date.today()) + '.xlsx', index = False)
    new_df.to_csv(homePop, index = False)

In [4]:
def automateKirkland(kirk_inv, curr_inventory):
    kirkland_df = pd.read_excel(kirk_inv)
    kirkland_df['Vendor Part Number '] = kirkland_df['Vendor Part Number '].astype(str)
    
    realTime_df = pd.read_csv(curr_inventory)
    realTime_df = realTime_df[['Name', 'Location Available', 'Location']]
    realTime_df = realTime_df[realTime_df['Location'] == 'KFHQ']
    realTime_df['Name'] = realTime_df['Name'].astype(str)

    realTime_df.rename(columns = {'Name':'Vendor Part Number '}, inplace = True)
    updated_kirkland = pd.merge(kirkland_df, realTime_df, on = 'Vendor Part Number ', how = 'left')
    
    supply = updated_kirkland['Location Available']
    chino = updated_kirkland['Inventory']
    for x in range(len(chino)):
        if math.isnan(supply[x]) == False:

            if supply[x] <= 40:
                chino[x] = 0
            elif supply[x] <= 60:
                chino[x] = 10
            elif supply[x] <= 100:
                chino[x] = 20
            else:

                chino[x] = int(supply[x] * .3)
        else:
            chino[x] = 0;

    updated_kirkland['Inventory'] = chino
    updated_kirkland = updated_kirkland.iloc[: , 0:3]
    updated_kirkland.to_excel(kirkland, index = False)
    
    # we need to drop the first column for kirklands upload 
    updated_kirkland = updated_kirkland.iloc[: , 1:3]

    updated_kirkland.to_excel('.\\Kirkland\\updated_kirkland_' + str(date.today()) + '.xlsx', index= False)


In [5]:
def automateNFM(inventory, realTimeInventory):
    
    realTime_df = pd.read_csv(realTimeInventory)
    realTime_df = realTime_df[['Name', 'Location Available', 'Location']]
    realTime_df = realTime_df[realTime_df['Location'] == 'KFHQ']
    realTime_df['Name'] = realTime_df['Name'].astype(str)

    realTime_df.rename(columns = {'Name':20210329}, inplace = True)
    
    
    
    nfm_df = pd.read_excel(inventory)

    updated_nfm = pd.merge(nfm_df, realTime_df, on = 20210329, how = 'left')
    stock = updated_nfm['CA']
    availability = updated_nfm['Chino']
    supply = updated_nfm['Location Available']
    
    restock = updated_nfm['Unnamed: 10']


    given_date = date.today()
    future_date = given_date+ relativedelta(months=5)
    future_date = str(future_date).replace('-','')

    for x in range(len(stock)):
        if math.isnan(supply[x]) == False:
            #change supply
            if supply[x] <= 40:
                stock[x] = 0
            elif supply[x] <= 60:
                stock[x] = 10
            elif supply[x] <= 100:
                stock[x] = 20
            else:
                stock[x] = int(supply[x] * .3)
        else:
            stock[x] = 0;
            #change availability
        if availability[x] != 'DI':
            if stock[x] == 0:
                availability[x] = 'OS'
                if math.isnan(restock[x]):
                    restock[x] = future_date
            else:
                availability[x] = 'AV'

    updated_nfm['CA'] = stock
    updated_nfm['Chino'] = availability
    updated_nfm = updated_nfm.iloc[:,0:11]
    #updated_nfm.rename(columns = {'20210329':'20210329'}, inplace = True)
    
    #change the usa column to have two decimal points
    updated_nfm['USA'].round(decimals = 2)
    updated_nfm.to_csv('.\\NFM\\updated_NFM_' + str(date.today()) +'.csv', index = False)
    updated_nfm.to_excel(nfm, index = False)

In [6]:
def automateHomeDepot(inventory, realTime):
    #load real time data
    realTime_df = pd.read_csv(realTime)
    realTime_df = realTime_df[['Name', 'Location Available', 'Location']]
    realTime_df = realTime_df[realTime_df['Location'] == 'KFHQ']
    realTime_df['Name'] = realTime_df['Name'].astype(str)

    realTime_df.rename(columns = {'Name':'VENDOR SKU'}, inplace = True)

    homedepot_df = pd.read_excel(inventory)
    updated_homedepot = pd.merge(homedepot_df, realTime_df, on = 'VENDOR SKU', how = 'left')
    stock = updated_homedepot['QTY']
    availability = updated_homedepot['AVAILABLE']
    supply = updated_homedepot['Location Available']

    

    for x in range(len(stock)):
        if math.isnan(supply[x]) == False:
            #change supply
            if supply[x] <= 40:
                stock[x] = 0
            elif supply[x] <= 60:
                stock[x] = 10
            elif supply[x] <= 100:
                stock[x] = 20
            else:
                stock[x] = int(supply[x] * .3)
        else:
            stock[x] = 0;
            #change availability
        if stock[x] == 0:
            availability[x] = 'No'
        else:
            availability[x] = 'Yes'

    updated_homedepot['AVAILABLE'] = availability
    updated_homedepot['WareHouse 1 QTY'] = stock
    updated_homedepot['QTY'] = stock
    
    
    updated_homedepot = updated_homedepot.iloc[:,0:45]
    
    updated_homedepot.to_excel('.\\HomeDepot\\updatedHomeDepot_' + str(date.today()) + '.xlsx',index = False)
    updated_homedepot.to_excel(homeDepot, index = False)


In [7]:
realTime = 'realTimeInventory.csv'
homeDepot = 'HomeDepotInventory.xlsx'
homePop = 'HomePopInventory.csv'
nfm = 'nfmInventory.xlsx'
kirkland = 'KirklandInventory.xlsx'

cymax = 'cymax_inventory.csv'
#autoCymax(cymax,realTime)
automateHomeDepot(homeDepot, realTime)
automateNFM(nfm, realTime)

automateKirkland(kirkland,realTime)
automateHomePop(homePop,realTime)

,Name,Description,Location,Location On Hand,Location Committed,Location Available
0,001775972,"Leather Storage Bench with tray, Brown \r\n",KFHQ,105.0,NaN,105.0
1,001775972,"Leather Storage Bench with tray, Brown \r\n",KFHQ : KFHQQU,NaN,NaN,NaN
2,001775972,"Leather Storage Bench with tray, Brown \r\n",KFDS,NaN,NaN,NaN
3,001775972,"Leather Storage Bench with tray, Brown \r\n",KFNJ,NaN,NaN,NaN
4,001776000,"LE STRG BC TRY,BLK \r\n",KFHQ,104.0,NaN,104.0
...,...,...,...,...,...,...
2409,PARTS,PARTS,- None -,NaN,NaN,NaN
2410,PROMOTION,- None -,- None -,NaN,NaN,NaN
2411,PROVISION,- None -,- None -,NaN,NaN,NaN
2412,RETURN/DEFECTIVE,- None -,- None -,NaN,NaN,NaN


KeyError: 'Location Available'